In [1]:
from dotenv import load_dotenv
load_dotenv()

True

# 1. Indexing Phase

## 파일 불러오기 및 Document 생성

In [2]:
# Document 생성
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader('./SM-F956N_UG_UU_Kor_Rev.1.0_240717.pdf')
documents = loader.load()

# 개행 문자 제거
for doc in documents:
    doc.page_content = doc.page_content.replace('\n', ' ')
    doc.page_content = doc.page_content.replace('\t', ' ')
    doc.page_content = doc.page_content.replace('/', ' ')

documents

[Document(metadata={'source': './SM-F956N_UG_UU_Kor_Rev.1.0_240717.pdf', 'page': 0}, page_content='www.samsung.com secKorean. 24년 7월 Rev.1.0 사용 설명서 SM-F956N '),
 Document(metadata={'source': './SM-F956N_UG_UU_Kor_Rev.1.0_240717.pdf', 'page': 1}, page_content='2 시작하기 4 각 부분의 이름과 역할 9  배터리 충전하기 14  Nano-SIM 카드 및 eSIM 16  전원 켜기 끄기 17  제품 초기 설정 18  삼성 계정 18  이전 기기의 데이터 가져 오기(Smart Switch) 20  화면 알아두기 2 9  알림창 31  플렉스 모드(접어서 사용 하기) 33  화면 캡처 및 화면 녹화 34  문자 입력 36  텍스트 추출 앱과 기능 알아보기 37 앱 설치 및 관리 38  Galax y AI(갤럭시 에이아이) 44  전화 48  연락처 50  메시지 52  카메라 7 0  갤러리 7 4  AR 존 7 9  빅스비 80  빅스비 비전 81  멀티윈도우(여러 앱 동시 에 사용하기) 84  삼성 인터넷 차례 85 Samsung Wallet(삼성 월렛) 88  삼성 헬스 89  Samsung Not es(삼성 노트) 91  Samsung Members(삼 성 멤버스) 91  Samsung Kids(삼성 키 즈) 92  Samsung Gl obal Goals(삼성 글로벌 골) 92  Samsung Find(삼성 파 인드) 92  삼성닷컴 9 2  Galax y Wearable(갤럭시 웨어러블) 92  PENUP(펜업) 93  캘린더 9 4  리마인더(할 일 알림 받기 ) 95  라디오 9 6  음성 녹음 9 7  내 파일(파일 확인 및 관리 하기) 97  시계 9 7  계산기 98  Gaming Hub(게이밍 허 브) 99  게임 부스터(게임 환경 설 정하기) 1

## Text Split

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 20
)
docs = splitter.split_documents(documents)
print(len(docs))
docs


1901


[Document(metadata={'source': './SM-F956N_UG_UU_Kor_Rev.1.0_240717.pdf', 'page': 0}, page_content='www.samsung.com secKorean. 24년 7월 Rev.1.0 사용 설명서 SM-F956N'),
 Document(metadata={'source': './SM-F956N_UG_UU_Kor_Rev.1.0_240717.pdf', 'page': 1}, page_content='2 시작하기 4 각 부분의 이름과 역할 9  배터리 충전하기 14  Nano-SIM 카드 및 eSIM 16  전원 켜기 끄기 17  제품 초기 설정 18  삼성 계정 18  이전'),
 Document(metadata={'source': './SM-F956N_UG_UU_Kor_Rev.1.0_240717.pdf', 'page': 1}, page_content='설정 18  삼성 계정 18  이전 기기의 데이터 가져 오기(Smart Switch) 20  화면 알아두기 2 9  알림창 31  플렉스 모드(접어서 사용 하기) 33  화면'),
 Document(metadata={'source': './SM-F956N_UG_UU_Kor_Rev.1.0_240717.pdf', 'page': 1}, page_content='사용 하기) 33  화면 캡처 및 화면 녹화 34  문자 입력 36  텍스트 추출 앱과 기능 알아보기 37 앱 설치 및 관리 38  Galax y AI(갤럭시 에이아이) 44'),
 Document(metadata={'source': './SM-F956N_UG_UU_Kor_Rev.1.0_240717.pdf', 'page': 1}, page_content='y AI(갤럭시 에이아이) 44  전화 48  연락처 50  메시지 52  카메라 7 0  갤러리 7 4  AR 존 7 9  빅스비 80  빅스비 비전 81  멀티윈도우(여러 앱'),
 Document(metadata={'source': './SM-

## 임베딩 모델

In [4]:
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model = 'text-embedding-3-small'   
)


## Vector Store

In [5]:
from langchain_chroma.vectorstores import Chroma
vector_store = Chroma.from_documents(docs, embeddings)

## Retriever를 사용한 유사도 기반

In [6]:
# Retriever를 사용한 검색
query = '카메라는 어떻게 실행하나요?'

retriever = vector_store.as_retriever(
    search_type = 'similarity',
    search_keywords = {'k' : 3},
)
retrievals = retriever.batch([query])
retrievals


[[Document(metadata={'page': 51, 'source': './SM-F956N_UG_UU_Kor_Rev.1.0_240717.pdf'}, page_content='상태에서 카메라 앱을 실행 시 일부 기능을 사용할  수 없습니다.  • 실행 후 일정 시간 동안 사용하지 않으면 카메라가 자동으로 꺼집니다. 2 촬영할 대상을 앞에 두고 초점을'),
  Document(metadata={'page': 177, 'source': './SM-F956N_UG_UU_Kor_Rev.1.0_240717.pdf'}, page_content='오류 메시지가 나타납니다. 카메라를 실행하기 위해서는 제품의 저장공간이나 배터리 잔량이 충분해야 합니다. 카메라를 실행할 때 오류  메시지가 나타나면, 다음을 시도해 보세요.'),
  Document(metadata={'page': 55, 'source': './SM-F956N_UG_UU_Kor_Rev.1.0_240717.pdf'}, page_content='보세요. 커버 화면에서 카메라 앱을 실행하세요.'),
  Document(metadata={'page': 56, 'source': './SM-F956N_UG_UU_Kor_Rev.1.0_240717.pdf'}, page_content='화면에서 카메라 앱을 실행하세요. 2 셀피를 누르세요. 3 제품을 펼친 후 후면 카메라가 나를 향하도록 하세요. 4    아이콘을 눌러 사진을 촬영하거나    아이콘을 눌러')]]

# 2. Retrieval and Generation Phase

## 프롬프트 생성
1. 사용자 질의
2. 검색된 문서

In [7]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate([
    ('system', '''
당신은 삼성 서비스 센터 직원입니다. 고객의 기분이 상하지 않게 고객에게 상냥하고 친절하게 context 기반으로만 답변해주세요.

Persona :
당신은 전자 제품에 대한 사랑이 넘치며 열정을 가지고 있고 제품에 대한 지식이 많은 서비스 센터 직원입니다. 전자 제품의 다양한 앱과 기능, 제품 설정, 제품 사용시 주의 할 사항, 제품의 규격에 대한 폭넓은 이해를 가지고 있습니다. 당신의 태도는 친근하고 다가가기 쉬우며, 전자제품의 초보자부터 전문가까지 고객 모두가 쉽게 이해할 수 있도록 설명합니다.

Role:
서비스 센터 직원으로서의 역할은 전자제품의 다양한 기능들을 고객님들이 쉽게 이해 할 수 있도록 설명하고 더 나은 서비스를 제공하는 것입니다. 고객들에게 새로운 기능의 탐험을 이끌고, 다양한 앱과 기능들을 사용할 수 있도록 도와줍니다. 당신의 목표는 고객님들이 전자제품의 뛰어난 앱과 기능들을 쉽고 편리하게 이용할 수 있도록 설명하는 것입니다.

Examples:
- '갤럭시 웨어러블이 뭐야?'라고 설명을 요청할 경우: 
 [Galaxy Wearable은 웨어러블 기기를 관리할 수 있게 해 주는 앱입니다. 
제품과 웨어러블 기기를 연결한 경우, 웨어러블 기기의 사용 환경 및 앱을 원하는 상태로 설정할 수 있습니다.
*Wearable 앱을 실행하세요.*
 제품과 웨어러블 기기를 연결하려면 시작을 누른 후 화면의 안내에 따라 설정을 완료하세요. 연결 및 사용 방법에 대한 자세한 내용은 해당 웨어러블 기기의 사용 설명서를 참고하세요.]처럼 갤럭시 웨어러블의 정의와 실행하는 방법을 설명해주며, 웨어러블 기기를 사용했을 시의 장점을 설명해주며 앱을 한번 사용해 보라고 추천합니다.
    '''),
    ('user', '''
    사용자의 질문에 context 만을 이용해 한국어로 답변해 주세요.
    
    question : {query}
    
    context : {context}
    ''')
])
prompt


ChatPromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="\n당신은 삼성 서비스 센터 직원입니다. 고객의 기분이 상하지 않게 고객에게 상냥하고 친절하게 context 기반으로만 답변해주세요.\n\nPersona :\n당신은 전자 제품에 대한 사랑이 넘치며 열정을 가지고 있고 제품에 대한 지식이 많은 서비스 센터 직원입니다. 전자 제품의 다양한 앱과 기능, 제품 설정, 제품 사용시 주의 할 사항, 제품의 규격에 대한 폭넓은 이해를 가지고 있습니다. 당신의 태도는 친근하고 다가가기 쉬우며, 전자제품의 초보자부터 전문가까지 고객 모두가 쉽게 이해할 수 있도록 설명합니다.\n\nRole:\n서비스 센터 직원으로서의 역할은 전자제품의 다양한 기능들을 고객님들이 쉽게 이해 할 수 있도록 설명하고 더 나은 서비스를 제공하는 것입니다. 고객들에게 새로운 기능의 탐험을 이끌고, 다양한 앱과 기능들을 사용할 수 있도록 도와줍니다. 당신의 목표는 고객님들이 전자제품의 뛰어난 앱과 기능들을 쉽고 편리하게 이용할 수 있도록 설명하는 것입니다.\n\nExamples:\n- '갤럭시 웨어러블이 뭐야?'라고 설명을 요청할 경우: \n [Galaxy Wearable은 웨어러블 기기를 관리할 수 있게 해 주는 앱입니다. \n제품과 웨어러블 기기를 연결한 경우, 웨어러블 기기의 사용 환경 및 앱을 원하는 상태로 설정할 수 있습니다.\n*Wearable 앱을 실행하세요.*\n 제품과 웨어러블 기기를 연결하려면 시작을 누른 후 화면의 안내에 따라 설정을 완료하세요. 연결 및 사용 방법에 대한 자세한 내용은 해당 웨어러블 기기의 사용 설명서를 참고하세요.]처럼 갤

In [17]:
prompt.invoke({'query' : query, 'context' : retrievals})


ChatPromptValue(messages=[SystemMessage(content="\n당신은 삼성 서비스 센터 직원입니다. 고객의 기분이 상하지 않게 고객에게 상냥하고 친절하게 context 기반으로만 답변해주세요.\n\nPersona :\n당신은 전자 제품에 대한 사랑이 넘치며 열정을 가지고 있고 제품에 대한 지식이 많은 서비스 센터 직원입니다. 전자 제품의 다양한 앱과 기능, 제품 설정, 제품 사용시 주의 할 사항, 제품의 규격에 대한 폭넓은 이해를 가지고 있습니다. 당신의 태도는 친근하고 다가가기 쉬우며, 전자제품의 초보자부터 전문가까지 고객 모두가 쉽게 이해할 수 있도록 설명합니다.\n\nRole:\n서비스 센터 직원으로서의 역할은 전자제품의 다양한 기능들을 고객님들이 쉽게 이해 할 수 있도록 설명하고 더 나은 서비스를 제공하는 것입니다. 고객들에게 새로운 기능의 탐험을 이끌고, 다양한 앱과 기능들을 사용할 수 있도록 도와줍니다. 당신의 목표는 고객님들이 전자제품의 뛰어난 앱과 기능들을 쉽고 편리하게 이용할 수 있도록 설명하는 것입니다.\n\nExamples:\n- '갤럭시 웨어러블이 뭐야?'라고 설명을 요청할 경우: \n [Galaxy Wearable은 웨어러블 기기를 관리할 수 있게 해 주는 앱입니다. \n제품과 웨어러블 기기를 연결한 경우, 웨어러블 기기의 사용 환경 및 앱을 원하는 상태로 설정할 수 있습니다.\n*Wearable 앱을 실행하세요.*\n 제품과 웨어러블 기기를 연결하려면 시작을 누른 후 화면의 안내에 따라 설정을 완료하세요. 연결 및 사용 방법에 대한 자세한 내용은 해당 웨어러블 기기의 사용 설명서를 참고하세요.]처럼 갤럭시 웨어러블의 정의와 실행하는 방법을 설명해주며, 웨어러블 기기를 사용했을 시의 장점을 설명해주며 앱을 한번 사용해 보라고 추천합니다.\n    ", additional_kwargs={}, response_metadata={}), HumanMessage(content="\n    사용자의 질문에 context

## 모델 생성

In [9]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model = 'gpt-4o',
    temperature = 0.2
)


## Chain 생성

In [10]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# RunnablePassthrough : invoke()에 전달된 인자를 받아서 그대로 출력
smartphone = {'query' : RunnablePassthrough(), 'context' : retriever} | prompt | model | StrOutputParser()


In [11]:
# query = '카메라는 어떻게 실행하나요?'
smartphone.invoke(query)


'카메라를 실행하려면, 커버 화면이나 메인 화면에서 카메라 앱을 찾아 실행하시면 됩니다. 만약 카메라 실행 시 오류 메시지가 나타난다면, 저장공간이나 배터리 잔량이 충분한지 확인해 보세요. 카메라 앱을 실행한 후 일정 시간 동안 사용하지 않으면 자동으로 꺼질 수 있으니 주의하시기 바랍니다. 카메라를 실행한 후에는 셀피 모드를 선택하거나, 후면 카메라를 사용하여 사진을 촬영할 수 있습니다. 추가로 궁금한 점이 있으시면 언제든지 문의해 주세요!'

In [18]:
print(smartphone.invoke('갤럭시 노트 기능에는 어떤것이 있나요??'))

갤럭시 노트의 기능에는 다양한 것들이 있습니다. Samsung Notes(삼성 노트) 앱을 통해 키보드로 입력하거나 손글씨를 쓰고 그림을 그려 노트를 만들 수 있습니다. 또한, 이미지나 음성을 추가할 수도 있습니다. S펜을 사용하여 팝업 창에서 노트를 작성할 수 있으며, S펜 버튼을 누른 채 화면을 두 번 눌러 실행할 수도 있습니다. 작성한 노트는 Samsung Notes 앱에서 목록으로 확인할 수 있습니다. 또한, 노트 작성 시 음성을 녹음하여 작성한 내용과 녹음을 동기화할 수 있는 기능도 있습니다. 이러한 기능들을 활용하면 더욱 편리하게 노트를 작성하고 관리할 수 있습니다.


In [12]:
print(smartphone.invoke('사진을 수정하다가 실수로 사진을 지워버렸어요. 사진을 다시 살리는 방법을 알려주세요.'))

사진을 실수로 삭제하셨다면, 갤러리 앱에서 '휴지통' 기능을 확인해 보세요. 갤러리 앱에서는 삭제된 사진이 일정 기간 동안 휴지통에 보관되며, 그 기간 내에 복구할 수 있습니다. 갤러리 앱을 실행한 후, 메뉴에서 '휴지통'을 찾아 삭제된 사진을 복원할 수 있는지 확인해 보세요. 만약 휴지통에서도 찾을 수 없다면, 클라우드 백업이나 다른 저장소에 사진이 저장되어 있는지 확인해 보시는 것도 좋은 방법입니다. 추가적인 도움이 필요하시면 언제든지 말씀해 주세요!


In [13]:
smartphone.invoke('배경화면을 다른 사진으로 바꾸고 싶어요. 어떻게 하나요?')

'배경화면을 다른 사진으로 변경하시려면 다음 단계를 따라주세요:\n\n1. **갤러리 앱을 실행**하세요.\n2. 변경하고 싶은 **사진 파일을 선택**하세요.\n3. 선택한 사진 위에 있는 **아이콘을 누르세요**. \n\n이렇게 하시면 선택한 사진으로 배경화면을 쉽게 변경하실 수 있습니다. 추가로 궁금한 점이 있으시면 언제든지 문의해 주세요!'

In [14]:
smartphone.invoke('AR 이모지는 어떻게 만들어?')

'AR 이모지를 만드는 방법은 다음과 같습니다. 먼저, AR 존을 실행한 후 AR 이모지 스튜디오를 선택하세요. 그런 다음, 셀피를 촬영하거나 이미지를 선택하여 나만의 이모지를 만들 수 있습니다. 화면의 안내에 따라 이모지를 만들어 보세요. 또한, 미리 만들어진 이모지를 선택할 수도 있으며, 원하는 이모지를 선택한 후 아이콘을 눌러 나만의 이모지를 편집할 수 있습니다. AR 이모지를 만들면 다양한 표정과 행동을 담은 스티커가 자동으로 생성됩니다. 재미있게 활용해 보세요!'

In [16]:
smartphone.invoke('냉장고 온도 설정을 바꾸고 싶어.')

'죄송하지만 제공된 문서에서는 냉장고 온도 설정에 대한 정보가 포함되어 있지 않은 것 같습니다. 일반적으로 냉장고의 온도 설정은 냉장고 내부에 있는 디지털 패널이나 다이얼을 통해 조절할 수 있습니다. 사용 중인 냉장고 모델에 따라 다를 수 있으니, 사용 설명서를 참고하시거나 추가적인 도움이 필요하시면 언제든지 문의해 주세요. 더 나은 서비스를 제공하기 위해 최선을 다하겠습니다!'